In [1]:
#==============================================================================================
# 원래 vocab 파일에 신규 vocab 파일에 대해 중복제거해서 추가하기
#==============================================================================================
import konlpy
from konlpy.tag import Mecab
from tqdm.notebook import tqdm
from nltk import FreqDist
import numpy as np
import torch
from transformers import BertTokenizer

In [2]:
# new_vocab 파일을 불러옴.
new_vocab_out = './moco-vocab/moco-corpus-kowiki2022-32000-en-vocab.txt'

# 2차 불용어 정의
#stopwords=['##다', '##하', '있', '##고', '##로', '##한', '##적', '##되', '##었', '##_']
stopwords=['']

new_vocab = []
with open(new_vocab_out, 'r', encoding='utf-8') as f:
    data = f.read().split('\n')

for word in tqdm(data):
    if not word in stopwords:  # 2차 불용어는 제외
        new_vocab.append(word)
        
print(new_vocab[0:10])

  0%|          | 0/32001 [00:00<?, ?it/s]

['the', 'to', 'of', 'a', 'is', 'and', 'in', 'for', '##The', 'that']


In [3]:
# 원래 vocab 리스트로 읽어옴
#org_vocab_out = './moco-vocab/mdistilbertV1.1/vocab.txt'
org_vocab_out = './moco-vocab/moco-corpus-kowiki2022-nouns-42000-Deduplication-149547.txt'

org_vocab = []

with open(org_vocab_out, 'r', encoding='utf-8') as f:
    data = f.read().split('\n')
    
for vocab in tqdm(data):
     org_vocab.append(vocab)

print(len(org_vocab))
print(org_vocab[1111:1115])

  0%|          | 0/149547 [00:00<?, ?it/s]

149547
['વ', 'શ', 'ષ', 'સ']


In [4]:
# 신규 vocab 리스트에서 원본 vocab 리스트 중복값을 제거
#temp = list(set(new_vocab) - set(org_vocab)) #순서 보존이 안됨
    
#또는

s = set(org_vocab)
temp = [x for x in new_vocab if x not in s] #순서 보존됨    

vocablist = org_vocab + temp  #원본 리스트 + 중복제가한 신규리스트 2개의 리스트를 합침

print(len(vocablist))
print(vocablist[1111:1115])

172523
['વ', 'શ', 'ષ', 'સ']


In [6]:
# 리스트를 파일로 저장
outfilepath = './moco-vocab/moco-corpus-kowiki2022-nouns-42000-Deduplication-add-en-159547.txt'
with open(outfilepath, 'w') as f:
    f.write('\n'.join(vocablist[:159547])) #159547개만 저장
    #f.write('\n'.join(vocablist))  #모두 저장
    
print('vocab 파일"{}" 추가 성공!. 출력 vocab 파일 : "{}"'.format(new_vocab_out, outfilepath))

vocab 파일"./moco-vocab/moco-corpus-kowiki2022-32000-en-vocab.txt" 추가 성공!. 출력 vocab 파일 : "./moco-vocab/moco-corpus-kowiki2022-nouns-42000-Deduplication-add-en-159547.txt"
